In [14]:
from common import *

cursor = connect()

# 视图

视图是存储在 PostgreSQL 数据库服务器中的命名查询。视图是基于一个或多个称为基表的表来定义的，定义视图的查询称为定义查询。创建视图后，可以像查询普通表一样查询视图中的数据。在幕后，PostgreSQL 会根据视图及其定义查询重写查询，执行查询后从基础表中获取数据。

除 `materialized views` 外，视图不存储数据。在 PostgreSQL 中，您可以创建称为物化视图的特殊视图，这些视图以物理方式存储数据，并定期从基础表中刷新数据。`materialized views` 在各种情况下都很方便，它能更快地访问远程服务器上的数据，并作为一种有效的缓存机制。

## 优点
- 简化复杂查询（Simplifying complex queries）
- 安全和访问控制（Security and access control）
- 逻辑数据独立性（Logical data independence）

# Create view

语法
```
CREATE VIEW view_name 
AS 
  query;
```

## Basic view

In [2]:
sql = """
CREATE VIEW contact AS 
SELECT 
  first_name, 
  last_name, 
  email 
FROM 
  customer;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1356441d3d0>

In [3]:
sql = """
SELECT * FROM contact;
"""
run_sql(cursor, sql)

    first_name  last_name                                  email
0        Jared        Ely           jared.ely@sakilacustomer.org
1         Mary      Smith          mary.smith@sakilacustomer.org
2     Patricia    Johnson    patricia.johnson@sakilacustomer.org
3        Linda   Williams      linda.williams@sakilacustomer.org
4      Barbara      Jones       barbara.jones@sakilacustomer.org
..         ...        ...                                    ...
594   Terrence  Gunderson  terrence.gunderson@sakilacustomer.org
595    Enrique   Forsythe    enrique.forsythe@sakilacustomer.org
596    Freddie     Duggan      freddie.duggan@sakilacustomer.org
597       Wade   Delvalle       wade.delvalle@sakilacustomer.org
598     Austin    Cintron      austin.cintron@sakilacustomer.org

[599 rows x 3 columns]


## Complex query

In [4]:
sql = """
CREATE VIEW customer_info AS 
SELECT 
  first_name, 
  last_name, 
  email, 
  phone, 
  city, 
  postal_code,
  country
FROM 
  customer 
  INNER JOIN address USING (address_id) 
  INNER JOIN city USING (city_id) 
  INNER JOIN country USING (country_id);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1356441d3d0>

In [5]:
sql = """
SELECT * FROM customer_info;
"""
run_sql(cursor, sql)

    first_name  last_name                                  email  \
0        Jared        Ely           jared.ely@sakilacustomer.org   
1         Mary      Smith          mary.smith@sakilacustomer.org   
2     Patricia    Johnson    patricia.johnson@sakilacustomer.org   
3        Linda   Williams      linda.williams@sakilacustomer.org   
4      Barbara      Jones       barbara.jones@sakilacustomer.org   
..         ...        ...                                    ...   
594   Terrence  Gunderson  terrence.gunderson@sakilacustomer.org   
595    Enrique   Forsythe    enrique.forsythe@sakilacustomer.org   
596    Freddie     Duggan      freddie.duggan@sakilacustomer.org   
597       Wade   Delvalle       wade.delvalle@sakilacustomer.org   
598     Austin    Cintron      austin.cintron@sakilacustomer.org   

            phone            city postal_code        country  
0     35533115997      Purwakarta       25972      Indonesia  
1     28303384290          Sasebo       35200          Ja

## 基于其他视图

In [6]:
sql = """
CREATE VIEW customer_usa 
AS 
SELECT 
  * 
FROM 
  customer_info 
WHERE 
  country = 'United States';
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1356441d3d0>

In [7]:
sql = """
SELECT * FROM customer_usa;
"""
run_sql(cursor, sql)

   first_name   last_name                                 email         phone  \
0     Zachary        Hite       zachary.hite@sakilacustomer.org  191958435142   
1     Richard     Mccrary    richard.mccrary@sakilacustomer.org  262088367001   
2       Diana   Alexander    diana.alexander@sakilacustomer.org    6171054059   
3       Scott     Shelley      scott.shelley@sakilacustomer.org  165450987037   
4     Clinton      Buford     clinton.buford@sakilacustomer.org  484500282381   
5       Wilma    Richards     wilma.richards@sakilacustomer.org  168758068397   
6     Valerie       Black      valerie.black@sakilacustomer.org  885899703621   
7        Karl        Seal          karl.seal@sakilacustomer.org  214756839122   
8       Betty       White        betty.white@sakilacustomer.org  517338314235   
9         Eva       Ramos          eva.ramos@sakilacustomer.org    9099941466   
10      Renee        Lane         renee.lane@sakilacustomer.org  662227486184   
11      Bryan    Hardison   

## Replace a view

In [8]:
sql = """
CREATE OR REPLACE VIEW contact AS 
SELECT 
  first_name, 
  last_name, 
  email,
  phone
FROM 
  customer
INNER JOIN address USING (address_id);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1356441d3d0>

In [9]:
sql = """
SELECT * FROM contact;
"""
run_sql(cursor, sql)

    first_name  last_name                                  email         phone
0        Jared        Ely           jared.ely@sakilacustomer.org   35533115997
1         Mary      Smith          mary.smith@sakilacustomer.org   28303384290
2     Patricia    Johnson    patricia.johnson@sakilacustomer.org  838635286649
3        Linda   Williams      linda.williams@sakilacustomer.org  448477190408
4      Barbara      Jones       barbara.jones@sakilacustomer.org  705814003527
..         ...        ...                                    ...           ...
594   Terrence  Gunderson  terrence.gunderson@sakilacustomer.org  935952366111
595    Enrique   Forsythe    enrique.forsythe@sakilacustomer.org  199514580428
596    Freddie     Duggan      freddie.duggan@sakilacustomer.org  644021380889
597       Wade   Delvalle       wade.delvalle@sakilacustomer.org  145308717464
598     Austin    Cintron      austin.cintron@sakilacustomer.org  288241215394

[599 rows x 4 columns]


# Drop view

语法
```
DROP VIEW [IF EXISTS] view_name 
[CASCADE | RESTRICT];
```

删除多个视图
```
DROP VIEW [IF EXISTS] view_name1, view_name2, ...
[CASCADE | RESTRICT];
```

In [15]:
sql = """
CREATE VIEW film_info AS 
SELECT 
  film_id, 
  title, 
  release_year, 
  length, 
  name category 
FROM 
  film 
  INNER JOIN film_category USING (film_id) 
  INNER JOIN category USING(category_id);


CREATE VIEW horror_film AS 
SELECT 
  film_id, 
  title, 
  release_year, 
  length 
FROM 
  film_info 
WHERE 
  category = 'Horror';


CREATE VIEW comedy_film AS 
SELECT 
  film_id, 
  title, 
  release_year, 
  length 
FROM 
  film_info 
WHERE 
  category = 'Comedy';


CREATE VIEW film_category_stat AS 
SELECT 
  name, 
  COUNT(film_id) 
FROM 
  category 
  INNER JOIN film_category USING (category_id) 
  INNER JOIN film USING (film_id) 
GROUP BY 
  name;


CREATE VIEW film_length_stat AS 
SELECT 
  name, 
  SUM(length) film_length 
FROM 
  category 
  INNER JOIN film_category USING (category_id) 
  INNER JOIN film USING (film_id) 
GROUP BY 
  name;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

In [11]:
sql = """
DROP VIEW comedy_film;
"""
cursor.execute(sql)

ProgrammingError: the last operation didn't produce a result

In [12]:
sql = """
DROP VIEW film_info;
"""
cursor.execute(sql)

DependentObjectsStillExist: 无法删除 视图 film_info 因为有其它对象倚赖它
DETAIL:  视图 horror_film 倚赖于 视图 film_info
HINT:  使用 DROP .. CASCADE 把倚赖对象一并删除.

In [16]:
sql = """
DROP VIEW film_info
CASCADE;
"""
cursor.execute(sql)

串联删除2个其它对象


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

In [17]:
sql = """
DROP VIEW film_length_stat, film_category_stat;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

# Updatable views

用于修改底层表中的数据。要使视图可更新，必须满足以下条件：
- 首先，视图的定义查询必须在 FROM 子句中包含一个条目，这个条目可以是一个表，也可以是另一个可更新的视图。
- 其次，定义查询的顶级子句不得包含以下子句之一：
    - GROUP BY
    - HAVING
    - LIMIT
    - DISTINCT
    - OFFSET FETCH
    - WITH
    - UNION
    - INTERSECT
    - EXCEPT
- 最后，定义查询的选择列表中不能包含任何内容：
    - Windows functions
    - Set-returning functions
    - Aggregate functions


对可更新视图执行 INSERT、UPDATE 或 DELETE 等修改语句时，PostgreSQL 会将该语句转换为底层表的相应语句。如果在视图的定义查询中设置了 WHERE 条件，则仍然可以通过视图更新或删除不可见的记录。不过，如果想避免这种情况，可以使用 WITH CHECK 选项来定义视图。

In [18]:
sql = """
CREATE TABLE cities (
    id SERIAL PRIMARY KEY ,
    name VARCHAR(255),
    population INT,
    country VARCHAR(50)
);

INSERT INTO cities (name, population, country)
VALUES
    ('New York', 8419600, 'US'),
    ('Los Angeles', 3999759, 'US'),
    ('Chicago', 2716000, 'US'),
    ('Houston', 2323000, 'US'),
    ('London', 8982000, 'UK'),
    ('Manchester', 547627, 'UK'),
    ('Birmingham', 1141816, 'UK'),
    ('Glasgow', 633120, 'UK'),
    ('San Francisco', 884363, 'US'),
    ('Seattle', 744955, 'US'),
    ('Liverpool', 498042, 'UK'),
    ('Leeds', 789194, 'UK'),
    ('Austin', 978908, 'US'),
    ('Boston', 694583, 'US'),
    ('Manchester', 547627, 'UK'),
    ('Sheffield', 584853, 'UK'),
    ('Philadelphia', 1584138, 'US'),
    ('Phoenix', 1680992, 'US'),
    ('Bristol', 463377, 'UK'),
    ('Detroit', 673104, 'US');

SELECT * FROM cities;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

### 创建可更新视图

In [19]:
sql = """
CREATE VIEW city_us 
AS 
SELECT 
  * 
FROM 
  cities 
WHERE 
  country = 'US';
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

In [21]:
sql = """
INSERT INTO city_us(name, population, country)
VALUES ('San Jose', 983459, 'US');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

In [22]:
sql = """
SELECT * FROM cities
WHERE name = 'San Jose';
"""
run_sql(cursor, sql)

   id      name  population country
0  21  San Jose      983459      US
1  22  San Jose      983459      US


In [23]:
sql = """
UPDATE city_us
SET population = 1000000
WHERE name = 'New York';
"""
cursor.execute(sql)

ProgrammingError: the last operation didn't produce a result

In [24]:
sql = """
SELECT * FROM cities 
WHERE name = 'New York';
"""
run_sql(cursor, sql)

   id      name  population country
0   1  New York     1000000      US


## WITH CHECK OPTION

要确保通过视图进行的任何数据修改都符合视图定义中的某些条件，可以使用 `WITH CHECK OPTION `子句。通常，在使用 CREATE VIEW 语句创建视图时指定 `WITH CHECK OPTION`：
```
CREATE VIEW view_name AS
query
WITH CHECK OPTION;
```

Check 的范围：
- LOCAL
- CASCADED

`LOCAL` 作用域将检查选项的执行范围限制为当前视图。它不会对当前视图所依赖的其他视图执行检查。
```
CREATE VIEW view_name AS
query
WITH LOCAL CHECK OPTION;
```
`CASCADED` 作用域将校验选项的执行范围扩展到当前视图的所有依赖的视图。
```
CREATE VIEW view_name AS
query
WITH CASCADED CHECK OPTION;
```


### 基础例子

In [15]:
sql = """
CREATE TABLE employees (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    department_id INT,
    employee_type VARCHAR(20) 
       CHECK (employee_type IN ('FTE', 'Contractor'))
);

INSERT INTO employees (first_name, last_name, department_id, employee_type)
VALUES
    ('John', 'Doe', 1, 'FTE'),
    ('Jane', 'Smith', 2, 'FTE'),
    ('Bob', 'Johnson', 1, 'Contractor'),
    ('Alice', 'Williams', 3, 'FTE'),
    ('Charlie', 'Brown', 2, 'Contractor'),
    ('Eva', 'Jones', 1, 'FTE'),
    ('Frank', 'Miller', 3, 'FTE'),
    ('Grace', 'Davis', 2, 'Contractor'),
    ('Henry', 'Clark', 1, 'FTE'),
    ('Ivy', 'Moore', 3, 'Contractor');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x13bc8666e50>

In [26]:
sql = """
CREATE OR REPLACE VIEW fte AS 
SELECT 
  id, 
  first_name, 
  last_name, 
  department_id,
  employee_type
FROM 
  employees 
WHERE 
  employee_type = 'FTE';
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

In [29]:
sql = """
SELECT * FROM fte;
"""
run_sql(cursor, sql)

   id first_name last_name  department_id employee_type
0   1       John       Doe              1           FTE
1   2       Jane     Smith              2           FTE
2   4      Alice  Williams              3           FTE
3   6        Eva     Jones              1           FTE
4   7      Frank    Miller              3           FTE
5   9      Henry     Clark              1           FTE


In [28]:
sql = """
INSERT INTO fte(first_name, last_name, department_id, employee_type) 
VALUES ('John', 'Smith', 1, 'Contractor');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

上述语句执行成功，但是数据未被插入，因为 `fte` 中的 `employee_type` 为 `FTE`，不符合 `Contractor` 的条件。因此我们可以在 `fte` 视图中加上 `WITH CHECK OPTION`。

In [30]:
sql = """
CREATE OR REPLACE VIEW fte AS 
SELECT 
  id, 
  first_name, 
  last_name, 
  department_id,
  employee_type
FROM 
  employees 
WHERE 
  employee_type = 'FTE'
WITH CHECK OPTION;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x13565129cd0>

In [31]:
# 再次尝试插入 Contractor 失败。但是更新 fte 是可以的
sql = """
INSERT INTO fte(first_name, last_name, department_id, employee_type) 
VALUES ('John', 'Snow', 1, 'Contractor');
"""
cursor.execute(sql)

WithCheckOptionViolation: 新行违反了视图"fte"的检查选项
DETAIL:  失败, 行包含(12, John, Snow, 1, Contractor).

### LOCAL CHECK OPTION

In [38]:
sql = """
CREATE OR REPLACE VIEW fte AS 
SELECT 
  id, 
  first_name, 
  last_name, 
  department_id,
  employee_type
FROM 
  employees 
WHERE 
  employee_type = 'FTE';


CREATE OR REPLACE VIEW fte_1 
AS 
SELECT 
  id, 
  first_name, 
  last_name, 
  department_id, 
  employee_type 
FROM 
  fte 
WHERE 
  department_id = 1 
WITH LOCAL CHECK OPTION;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1356512af90>

In [39]:
sql = """
SELECT * FROM fte_1;
"""
run_sql(cursor, sql)

   id first_name last_name  department_id employee_type
0   1       John       Doe              1           FTE
1   6        Eva     Jones              1           FTE
2   9      Henry     Clark              1           FTE


PostgreSQL 只会检查 `fte_1` 视图。

### CASCADED CHECK OPTION

In [40]:
sql = """
CREATE OR REPLACE VIEW fte_1
AS 
SELECT 
  id, 
  first_name, 
  last_name, 
  department_id, 
  employee_type 
FROM 
  fte 
WHERE 
  department_id = 1 
WITH CASCADED CHECK OPTION;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1356512af90>

In [41]:
sql = """
INSERT INTO fte_1(first_name, last_name, department_id, employee_type) 
VALUES ('Peter', 'Taylor', 1, 'Contractor');
"""
cursor.execute(sql)

WithCheckOptionViolation: 新行违反了视图"fte"的检查选项
DETAIL:  失败, 行包含(11, Peter, Taylor, 1, Contractor).